Many thanks to nampdn-ai for the data set. 

https://huggingface.co/datasets/nampdn-ai/tinystories-vietnamese

Copyright Hoa Vu

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import random
import pickle
import math
from tokenizers.processors import BertProcessing
from transformers import AutoTokenizer, BertTokenizer
import gc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

C:\Users\hvutr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(device)

if not torch.cuda.is_available():
    print("CUDA is not available on your system.")
else:
    # Print the number of CUDA devices
    print(f"Number of CUDA Devices: {torch.cuda.device_count()}\n")

    for i in range(torch.cuda.device_count()):
        print(f"Device {i}:")
        print(f"    Name: {torch.cuda.get_device_name(i)}")
        print(f"    Computational Capability: {torch.cuda.get_device_capability(i)}")
        print(f"    Total Memory: {torch.cuda.get_device_properties(i).total_memory / (1024**2):.2f} MB\n")



cuda
Number of CUDA Devices: 1

Device 0:
    Name: NVIDIA GeForce RTX 3090
    Computational Capability: (8, 6)
    Total Memory: 24575.50 MB



In [3]:
data_path = 'datasets/tiny_stories/'

In [4]:
filename = 'tiny_stories.csv'
filepath = os.path.join(data_path, filename)

data = pd.read_csv(filepath)
data = data.dropna()

print(data.head())


   Unnamed: 0                                                 en  \
0           0  Once upon a time, there was a little boy named...   
1           1  Once upon a time, there was a normal boy named...   
2           2  Once upon a time, there was a fast car named S...   
3           3  Once there was a family with a very playful go...   
4           4  Once upon a time, there was a dog named Max. M...   

                                                  vi  
0  Ngày xửa ngày xưa, có một cậu bé tên Tim. Tim ...  
1  Ngày xửa ngày xưa, có một cậu bé bình thường t...  
2  Ngày xửa ngày xưa, có một chiếc xe tốc độ tên ...  
3  Có một gia đình có một con dê rất hay vui đùa....  
4  Ngày xửa ngày xưa, có một con chó tên là Max. ...  


In [9]:
N = len(data)
train_size = int(0.9 * N)
train_data = data[:train_size]
val_data = data[train_size:]

print(train_data.shape, val_data.shape)

(2339822, 3) (259981, 3)


In [5]:
from transformers import BertTokenizer, AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

# Get the IDs
cls_id = tokenizer.cls_token_id
sep_id = tokenizer.sep_token_id
unk_id = tokenizer.unk_token_id
pad_id = tokenizer.pad_token_id

print("PAD Token ID:", pad_id)
print("UNK Token ID:", unk_id)
print("CLS Token ID:", cls_id)
print("SEP Token ID:", sep_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


PAD Token ID: 1
UNK Token ID: 3
CLS Token ID: 0
SEP Token ID: 2


In [6]:
BLOCK_SIZE = 256
VOCAB_SIZE = tokenizer.vocab_size
print('Vocab size is ', VOCAB_SIZE)


Vocab size is  64000


In [10]:
def get_batch(data, block_size, batch_size):
    x = torch.zeros((batch_size, block_size), dtype=torch.long)
    y = torch.zeros((batch_size, block_size), dtype=torch.long)
    samples = data['vi'].sample(n=batch_size)

    for i, sample in enumerate(samples):
        token_ids = tokenizer.encode(sample)
        token_ids = [id for id in token_ids if id != unk_id]
        if len(token_ids) < block_size + 2:
            token_ids = token_ids + [pad_id] * (block_size + 2 - len(token_ids))
        random_start = random.randint(0, len(token_ids) - block_size - 2)
        x[i, :len(token_ids)] = torch.tensor(token_ids[random_start:random_start + block_size], dtype=torch.long)
        y[i, :len(token_ids)] = torch.tensor(token_ids[random_start + 1:random_start + block_size + 1], dtype=torch.long)

    return x, y

a, b = get_batch(train_data, block_size=BLOCK_SIZE, batch_size=1)
print(a.shape, b.shape)
print(tokenizer.decode(a[0].tolist(), skip_special_tokens=False))
print(tokenizer.decode(b[0].tolist(), skip_special_tokens=False))

torch.Size([1, 256]) torch.Size([1, 256])
<s> Ngày xửa ngày xưa, ở một vùng đất sống động có một cô bé tên Mia. Mia có người bạn thân tên Tim. Họ thích chơi đùa và cười đùa cùng nhau. Một ngày nọ, Mia và Tim cãi nhau."Không, tôi muốn chơi búp bê" Mia nói.'Không, con muốn chơi với xe' Tim nói.Mia và Tim rất buồn. Họ không biết phải làm gì. Rồi, một con chó ngoan đến với họ. Con chó thấy Mia và Tim đang buồn. Con chó muốn an ủi họ. Nó chơi búp bê lẫn xe hơi. Mia và Tim nhìn thấy con chó và bắt đầu cười."Hãy chơi cùng nhau như một chú chó" Mia nói.'Vâng, chúng ta hãy chơi và vui vẻ' Tim nói.Vì vậy, Mia và Tim chơi với búp bê và xe hơi. Họ lại vui vẻ. Con chó đã giúp họ trở thành bạn bè một lần nữa. Họ cảm ơn con chó và chơi đùa trên mảnh đất tươi đẹp, cười đùa và vui vẻ cùng nhau. </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Ngày xửa ngày xưa, ở một vùng đấ

In [ ]:
del model, xb, yb  # delete the tensor variable
torch.cuda.empty_cache()  # clear unused memory in PyTorch
gc.collect()  # call Python garbage collector

In [11]:
N_EMB = 800
N_LAYERS = 6
N_HEADS = 10
DROPOUT = 0.2


def estimate_loss(model, val_data, block_size, batch_size):
    model.eval()
    with torch.no_grad():
        x, y = get_batch(val_data, block_size, batch_size)
        x, y = x.to(device), y.to(device)
        _, loss = model(x, y)
    model.train()
    return loss.item()

def generate_square_subsequent_mask(sz):
    mask = (torch.tril(torch.ones(sz, sz)) == 1).float()
    mask = mask.masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def get_sine_position_encodings(length, dim):
    pos = torch.arange(length, dtype=torch.float32).reshape(-1, 1)
    div_term = torch.exp(torch.arange(0, dim, 2).float() * -(math.log(10000.0) / dim))
    pos_encodings = torch.zeros(length, dim)
    pos_encodings[:, 0::2] = torch.sin(pos * div_term)
    pos_encodings[:, 1::2] = torch.cos(pos * div_term)
    return pos_encodings

class LanguageModel(nn.Module):

    def __init__(self, vocab_size, n_emb, block_size, n_layers, n_heads, dropout=0.2):
        super(LanguageModel, self).__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, n_emb)
        self.position_embedding_table = nn.Embedding(block_size, n_emb)
        self.block_size = block_size

        encoder_layer = nn.TransformerEncoderLayer(d_model=n_emb, nhead=n_heads, dropout=dropout, layer_norm_eps=1e-6)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)

        self.feed_forward = nn.Sequential(
            nn.Linear(n_emb, 2 * n_emb),
            nn.ReLU(),
            nn.Linear(2 * n_emb, n_emb)
        )

        self.lm_head = nn.Linear(n_emb, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        token_emb = self.token_embedding_table(idx)
        position_emb = self.position_embedding_table(torch.arange(T, device=device))

        x = token_emb + position_emb
        x_transform = x.clone()
        mask = generate_square_subsequent_mask(T).to(device)

        x_transform = self.transformer_encoder(x_transform.permute(1, 0, 2), mask=mask)
        x_transform = x_transform.permute(1, 0, 2)
        x = x + x_transform

        x = self.feed_forward(x)
        logits = self.lm_head(x)

        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
            return logits, loss
        else:
            return logits, None


    def generate(self, idx, max_new_tokens, temperature=1.0, stop_token=False):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.block_size:]
            logits, _ = self.forward(idx_cond)

            # Scale logits by the temperature
            logits = logits[:, -1, :] / temperature

            probs = F.softmax(logits, dim=-1)
            idx_new = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, idx_new], dim=-1)
            if stop_token and idx_new.item() == sep_id:
                break
        return idx

# Create model, optimizer
model = LanguageModel(vocab_size=VOCAB_SIZE, block_size=BLOCK_SIZE, n_emb=N_EMB, n_layers=N_LAYERS, \
    n_heads=N_HEADS, dropout=DROPOUT).to(device)

print(f'Number of parameters {sum(p.numel() for p in model.parameters() if p.requires_grad)}')

C:\Users\hvutr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Number of parameters 140307488


In [13]:
model_path = os.path.join(data_path, "vietnamese_tiny_stories_6layers.pth")
print(model_path)
model = torch.load(model_path)

datasets/tiny_stories/vietnamese_tiny_stories_6layers.pth


In [12]:
EARLY_STOP = 50
N_EPOCHS = 10000
BATCH_SIZE = 32
LEARNING_RATE = 3e-4
SAVE_FREQ = 500
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

model_path = os.path.join(data_path, "vietnamese_tiny_stories_6layers.pth")

for steps in range(N_EPOCHS):
    model.train()
    xb, yb = get_batch(train_data, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE)
    xb = xb.to(device)
    yb = yb.to(device)
    logits, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Delete xb, yb and free GPU memory
    del xb, yb
    torch.cuda.empty_cache()

    if steps % 100 == 0:
        print('Step:', steps, 'Training Loss:', loss.item())
        val_loss = estimate_loss(model, val_data, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE)
        print('Validation loss:', val_loss)
    if steps % SAVE_FREQ == 0:
        torch.save(model, model_path)


Step: 0 Training Loss: 11.044466018676758
Validation loss: 9.7381010055542
Step: 100 Training Loss: 3.950052261352539
Validation loss: 4.015854835510254
Step: 200 Training Loss: 3.357128143310547
Validation loss: 3.2051987648010254
Step: 300 Training Loss: 3.2017900943756104
Validation loss: 3.228306293487549
Step: 400 Training Loss: 3.0364573001861572
Validation loss: 2.8704798221588135
Step: 500 Training Loss: 2.9219369888305664
Validation loss: 2.786396026611328
Step: 600 Training Loss: 2.45868182182312
Validation loss: 2.651737689971924
Step: 700 Training Loss: 2.5921571254730225
Validation loss: 2.533076047897339
Step: 800 Training Loss: 2.404451847076416
Validation loss: 2.6880455017089844
Step: 900 Training Loss: 2.3230371475219727
Validation loss: 2.366144895553589
Step: 1000 Training Loss: 2.3544352054595947
Validation loss: 2.2895309925079346
Step: 1100 Training Loss: 2.2965869903564453
Validation loss: 2.1147689819335938
Step: 1200 Training Loss: 2.2847282886505127
Validatio

In [17]:
model_path = os.path.join(data_path, "vietnamese_tiny_stories_6layers.pth")
torch.save(model, model_path)

In [14]:
starting_tokens = 'Ở một ngôi làng hẻo lánh, có một con quạ xấu xí. '
encoded_start = tokenizer.encode(starting_tokens.lower())
encoded_start.pop(-1)
len_starting_tokens = len(encoded_start)

idx = torch.tensor(encoded_start).reshape(1, len_starting_tokens).to(device)
model.eval()
N_SAMPLES = 20
for _ in range(N_SAMPLES):
    generation = model.generate(idx, max_new_tokens=2000, temperature=0.7, stop_token=True)[0].tolist()
    story = tokenizer.decode(generation, skip_special_tokens=False)

    print('Story ', _ + 1, ':')
    print(story)
    print('\n')



Story  1 :
<s> ở một ngôi làng hẻo lánh, có một con quạ xấu xí. Con quạ xấu xí không có bạn bè để chơi cùng. Một ngày nọ, con quạ xấu xí gặp một con chim nhỏ. Con chim bị lạc và cần được giúp đỡ.Con quạ xấu xí muốn giúp con chim tìm đường về nhà. Nó bay lên cao và nhìn thấy một tảng đá lớn. Con chim nói, "Xin chào hòn đá. Tại sao bạn lại buồn?" Hòn đá nói, "Tôi không thể tìm thấy mẹ tôi. Tôi không thể tìm được mẹ tôi." Con chim xinh đẹp muốn giúp đỡ. Con chim xinh đẹp nói, "Tôi sẽ giúp bạn tìm mẹ của bạn."Chẳng bao lâu sau, chú chim xinh đẹp tìm thấy con chim xinh đẹp trên tảng đá. Chú chim xinh đẹp và chú chim xinh đẹp đến chỗ tảng đá đó. Tảng đá cũng hạnh phúc nữa. Chú chim xinh xắn nói, "Cảm ơn, chú chim xinh xắn. Tôi sẽ giúp bạn." Con chim xinh đẹp mỉm cười và nói, "Không có gì, chú chim xinh đẹp nhỏ. Hãy mãi là bạn bè và họ sẽ luôn ở bên nhau." </s>


Story  2 :
<s> ở một ngôi làng hẻo lánh, có một con quạ xấu xí. Con quạ sống trên một cái cây lớn. Nó có nhiều bạn bè như chim chóc